In [ ]:
# KUC, NLP

3. SVD, truncated SVD


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# original data
sms = pd.read_csv('sms-spam.csv',usecols= ['spam','text'])
print(sms.head())

In [ ]:
# processed
index = ['sms{}{}'.format(i, '!'*j) for (i,j) in zip(range(len(sms)), sms.spam)]  # <2>
print(index)


In [ ]:
print(len(sms.values))

In [ ]:
sms = pd.DataFrame(sms.values, columns=sms.columns, index=index)
print(sms)

In [ ]:
mask = sms.spam.astype(bool).values
print(mask)

In [ ]:
sms['spam'] = sms.spam.astype(int)

print(sms.head())

In [ ]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize.casual import casual_tokenize

n_samples = 4837 # the number of samples in the data file
n_features = 100 # the size of vocabulary
n_components = 10 # number of topics
n_top_words = 20
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
)
tfidf = tfidf_vectorizer.fit_transform(sms.text)


In [ ]:
print(sms.text)
print(tfidf)

In [ ]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print(tfidf_feature_names)

## LSA, based on tf-idf

In [ ]:
# LSA: truncated SVD
import pandas as pd
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=10, n_iter=100)
svd_topic_vectors = svd.fit_transform(tfidf)
print(svd_topic_vectors.shape)

In [ ]:
svd_topic_vectors[0]

In [ ]:
topic_terms = svd.components_
topic_terms.shape

In [ ]:
print(topic_terms[0])

In [ ]:
svd_topic_vectors = pd.DataFrame(svd_topic_vectors)
svd_topic_vectors.round(3).head(16)


In [ ]:
topic_terms

In [ ]:
# illustrated topics after truncated SVD: display topics and terms
top_terms = 30
TOTAL_TOPICS = 10 # we assume the optimal number of topics
vocabulary = np.array(tfidf_feature_names)
topic_key_term_idxs = np.argsort(-np.absolute(topic_terms), axis=1)[:,:top_terms]
topic_keyterm_weights = np.array([topic_terms[row, columns] for row, columns in list(zip(np.arange(TOTAL_TOPICS), topic_key_term_idxs))])

In [ ]:
topic_keyterms = vocabulary[topic_key_term_idxs]
topic_keyterms_weights = list(zip(topic_keyterms, topic_keyterm_weights))


In [ ]:
# topic_terms.shape
print(topic_key_term_idxs[0])
print(tfidf_feature_names[topic_key_term_idxs[0]])
print(topic_keyterms_weights[0])

In [ ]:
for n in range(TOTAL_TOPICS):
    print('Topic #'+str(n+1)+':')
    print('='*50)
    d1 = []
    d2 = []
    terms, weights = topic_keyterms_weights[n]
    term_weights = sorted([(t, w) for t, w in zip(terms, weights)],
                          key=lambda row: -abs(row[1]))

    for term, wt in term_weights:
        if wt >= 0:
            d1.append((term, round(wt, 3)))
        else:
            d2.append((term,round(wt,3)))
    print('Direction 1:', d1)
    print('Direction 2:', d2)